In [1]:
import pandas as pd

import snowflake.connector

import numpy as np

%matplotlib inline

import matplotlib.pyplot as plt 

from pydataset import data

from datetime import datetime, timedelta

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

In [2]:
import config as cfg

In [3]:
snowflake_user = cfg.snowflake_access['snowflake_user']
snowflake_pass = cfg.snowflake_access['snowflake_pass']
snowflake_acct = cfg.snowflake_access['snowflake_acct']
snowflake_wh = 'BIRDFACTS_PROD_WAREHOUSE'
snowflake_db = 'BIRDFACTSDEV'
snowflake_schema = 'BIRDFACTS_ANALYTICS'

con = snowflake.connector.connect(
    user = snowflake_user,
    password = snowflake_pass,
    account = snowflake_acct,
    warehouse = snowflake_wh,
    database = snowflake_db,
    schema = snowflake_schema
)

cs = con.cursor()

In [35]:
def sales(start, end, material_family, gender):
    sales_query = """
    select
    happened_at_local_date,
    taxonomy_style,
    sum(gross_sales_usd) as gross_sales,
    sum(quantity) as unit_sales
    from
    fact_sales
    where
    sales_channel = 'eCommerce'
    and happened_at_local_date between """+ start +""" and """ + end + """
    and profit_center_country = 'United States'
    and taxonomy_category = 'Shoes'
    and material_family = """ + material_family + """
    and gender in """  + gender + """
    and event_type = 'order'
    group by 1, 2
    order by 1, 2
    """
    sales = pd.DataFrame(cs.execute(sales_query).fetchall(),
                           columns = ([col[0] for col in cs.description]))
    sales['GROSS_SALES'] = pd.to_numeric(sales['GROSS_SALES'])
    sales['UNIT_SALES'] = pd.to_numeric(sales['UNIT_SALES'])
    sales_pivot = sales.pivot_table(index = 'HAPPENED_AT_LOCAL_DATE',
                 columns = 'TAXONOMY_STYLE',
                 values = 'UNIT_SALES',
                 aggfunc = 'sum').reset_index().fillna(0)
    sales_pivot.to_clipboard(index = False)

In [36]:
sales("'2020-04-01'", "'2020-05-31'", "'Tree'", "('Male', 'Female')")

In [37]:
sales("'2019-04-01'", "'2019-05-31'", "'Tree'", "('Male')")

In [38]:
sales("'2019-04-01'", "'2019-05-31'", "'Tree'", "('Male', 'Female')")

In [33]:
sales("'2020-04-01'", "'2020-05-31'", "'Wool'")